In [1]:
%load_ext ipyreact

In [2]:
# To convert FontAwesome icons into API icons.

import xml.etree.ElementTree as ET

svg_string = """
<svg xmlns="http://www.w3.org/2000/svg" height="1em" viewBox="0 0 512 512"><path d="M416 208c0 45.9-14.9 88.3-40 122.7L502.6 457.4c12.5 12.5 12.5 32.8 0 45.3s-32.8 12.5-45.3 0L330.7 376c-34.4 25.2-76.8 40-122.7 40C93.1 416 0 322.9 0 208S93.1 0 208 0S416 93.1 416 208zM136 184c-13.3 0-24 10.7-24 24s10.7 24 24 24H280c13.3 0 24-10.7 24-24s-10.7-24-24-24H136z"/></svg>
"""

root = ET.fromstring(svg_string)
path = root.find("svg:path", {'svg': 'http://www.w3.org/2000/svg'})
print({"xmlns": "http://www.w3.org/2000/svg", **root.attrib, **path.attrib})


{'xmlns': 'http://www.w3.org/2000/svg', 'height': '1em', 'viewBox': '0 0 512 512', 'd': 'M416 208c0 45.9-14.9 88.3-40 122.7L502.6 457.4c12.5 12.5 12.5 32.8 0 45.3s-32.8 12.5-45.3 0L330.7 376c-34.4 25.2-76.8 40-122.7 40C93.1 416 0 322.9 0 208S93.1 0 208 0S416 93.1 416 208zM136 184c-13.3 0-24 10.7-24 24s10.7 24 24 24H280c13.3 0 24-10.7 24-24s-10.7-24-24-24H136z'}


In [3]:
import ipyreact
import sqlalchemy
import uuid
import glob
import os
import pathlib
import pandas
import mimetypes
import traitlets
from PIL import Image as PIL_Image
from IPython.display import display, HTML

class WhimsyAnnotator(ipyreact.ReactWidget):
    img_path = traitlets.Unicode().tag(sync=True)
    img_mimetype = traitlets.Unicode("").tag(sync=True)
    img_data = traitlets.Bytes().tag(sync=True)
    img_w = traitlets.Int(1).tag(sync=True)
    img_h = traitlets.Int(1).tag(sync=True)

    labels = traitlets.List().tag(sync=True)
    annotations = traitlets.List().tag(sync=True)

    @traitlets.observe("annotations")
    def onAnnotationsChange(self, change):
        pass
    
    def __init__(self, **kwargs):
        super(type(self), self).__init__(**kwargs)

        self.labels = ["red", "blue", "green", "yellow"]
        self.annotations = [
            {"x": 0.2, "y": 0.1, "w": 0.5, "h": 0.5, "confidence": None, "type": "gt", "label": "blue", "uuid": "05823aca-7d8b-4a40-a2a2-076cf29bc464"},
            {"x": 0.0, "y": 0.1, "w": 0.1, "h": 0.6, "confidence": None, "type": "gt", "label": "red", "uuid": "2c052daa-799b-495a-a90c-86c3a30823fe"},
            {"x": 0.5, "y": 0.3, "w": 0.3, "h": 0.4, "confidence": None, "type": "gt", "label": "green", "uuid": "4cf76cf0-4f8c-4eea-bc2a-488942613c70"}
        ]

    def loadFile(self, filepath):
        file_as_path = pathlib.Path(filepath)
        assert file_as_path.is_file()
        self.img_path = file_as_path.absolute().as_posix()
        self.img_mimetype = mimetypes.guess_type(file_as_path)[0]
        self.img_data = file_as_path.read_bytes()
        imgdetails = PIL_Image.open(file_as_path)
        self.img_w, self.img_h = imgdetails.size
        imgdetails.close()
    
    _esm = pathlib.Path("whimsy.js")
    _css = pathlib.Path("whimsy.css").read_text()

display(HTML("<style>" + WhimsyAnnotator._css + "</style>"))

w = WhimsyAnnotator()
w


WhimsyAnnotator(annotations=[{'x': 0.2, 'y': 0.1, 'w': 0.5, 'h': 0.5, 'confidence': None, 'type': 'gt', 'label…

In [4]:
w.loadFile("sans_mortadela.jpg")